# Task 1.3 | Input a word, return a tuple

In [2]:
import re
def findloc(stuff):
    two=sc.textFile('taleOfTwoCities.txt')
    two_index=two.zipWithIndex()
    line=two_index.collect()
    words=two_index.flatMap(lambda (k,v): [(word,v) for word in k.split()])
    if len(stuff.split()) != 1:
        print("Input only takes 1 word")
    else:
        for i in words.collect():
            if bool(re.search(stuff,i[0])):
                for j in line:
                    if j[1] == i[1]:
                        print(i[1]+1,j[0].find(stuff))   

In [3]:
findloc('wisdom')

(3, 18)
(2311, 43)


In [4]:
from pyspark.sql import Row
from datetime import datetime

# Define functions to parse txt files containing itmes, stores, customers, and transactions
def parseStore(s):
    l=s.split('|')
    return Row(store_num=int(l[0]), 
               store_name=l[1],               
               store_zone=l[2],
               store_city=l[3], 
               store_state=l[4], 
               store_type=int(l[5]))

def parseItem(s):
    l=s.split('|')
    return Row(item_number=int(l[0]), 
               dept_categ_class=l[1],               
               item_des=l[2],
               item_unt_qty=float(l[3]), 
               size_unit_desc=l[4], 
               brand_code=l[5], 
               dept_num=int(l[6]), 
               dept_name=l[7], 
               categ_num=int(l[8]), 
               categ_name=l[9], 
               class_num=int(l[10]),
               class_name=l[11])

def parseCustomer(s):
    l=s.split('|')
    return Row(hshld_acct=int(l[0]),
               birth_yr_head_hh=l[1],
               hh_income=l[2],
               hh_size=l[3],
               adult_count=l[4],
               child_count=l[5],
               birth_yr_oldest=l[6],
               birth_yr_youngest=l[7],
               bad_address=l[8],
               privacy=l[9],
               application_date=datetime.strptime(l[10],'%Y-%m-%d'),
               wine_email_sent=int(l[11]),
               wine_email_open=int(l[12]),
               wine_email_click=int(l[13]))

def parsePostrans(s):
    l=s.split('|')
    return Row(hshld_acct=int(l[0]),
               acct_num=int(l[1]),
               trans_num=int(l[2]),
               trans_date=datetime.strptime(l[3],'%Y-%m-%d'),
               store_num=int(l[4]),
               item_number=int(l[5]),
               dept_categ_class=l[6],
               unit_count=int(l[7]),
               net_sales=float(l[8]),
               gross_sales=float(l[9]),
               manuf_coupon=float(l[10]))

In [5]:
path='/public/tbiswas2/csc261/spark/wegmans/'

storeDF=spark.createDataFrame(sc.textFile(path+'wegmans_store_master.txt').map(lambda x: parseStore(x)))
customerDF=spark.createDataFrame(sc.textFile(path+'wegmans_customer_master.txt').map(lambda x: parseCustomer(x)))
itemDF=spark.createDataFrame(sc.textFile(path+'wegmans_item_master.txt').map(lambda x: parseItem(x)))
postransDF=spark.createDataFrame(sc.textFile(path+'partial_transaction.dat').map(lambda x: parsePostrans(x)))

In [6]:
storeDF.show(5)
customerDF.show(5)
itemDF.show(5)
postransDF.show(5)

+-------------+--------------------+---------+-----------+----------+----------+
|   store_city|          store_name|store_num|store_state|store_type|store_zone|
+-------------+--------------------+---------+-----------+----------+----------+
|        DEPEW|   WEGMANS DICK ROAD|       80|         NY|         1|   BUFFALO|
|      AMHERST|WEGMANS ALBERTA D...|       82|         NY|         1|   BUFFALO|
|WILLIAMSVILLE|WEGMANS SHERIDAN ...|       83|         NY|         1|   BUFFALO|
|      BUFFALO|    WEGMANS MCKINLEY|       84|         NY|         1|   BUFFALO|
|      AMHERST|WEGMANS NIAGARA F...|       86|         NY|         1|   BUFFALO|
+-------------+--------------------+---------+-----------+----------+----------+
only showing top 5 rows

+-----------+--------------------+-----------+----------------+---------------+-----------------+-----------+---------+-------+----------+-------+----------------+---------------+---------------+
|adult_count|    application_date|bad_address|birt

In [7]:
storeDF.show(2)

+----------+--------------------+---------+-----------+----------+----------+
|store_city|          store_name|store_num|store_state|store_type|store_zone|
+----------+--------------------+---------+-----------+----------+----------+
|     DEPEW|   WEGMANS DICK ROAD|       80|         NY|         1|   BUFFALO|
|   AMHERST|WEGMANS ALBERTA D...|       82|         NY|         1|   BUFFALO|
+----------+--------------------+---------+-----------+----------+----------+
only showing top 2 rows



# Task 2.3        |           50 stores in the sample are in NY state

In [8]:
storeDF.groupBy('store_state').count().show()

+-----------+-----+
|store_state|count|
+-----------+-----+
|         NJ|   12|
|         VA|    6|
|         MD|    8|
|         PA|   16|
|         NY|   50|
|         MA|    2|
+-----------+-----+



In [9]:
itemDF.show(3)

+----------+--------------------+---------+--------------------+---------+----------------+---------+--------+------------------+-----------+------------+--------------+
|brand_code|          categ_name|categ_num|          class_name|class_num|dept_categ_class|dept_name|dept_num|          item_des|item_number|item_unt_qty|size_unit_desc|
+----------+--------------------+---------+--------------------+---------+----------------+---------+--------+------------------+-----------+------------+--------------+
|      WEGF|WEGMANS WHITE BREADS|        1|  WEGMAN WHITE BREAD|        1|          010101| BAKESHOP|       1|    WG GIANT BREAD|      19190|        22.0|            OZ|
|      WEGF|WEGMANS WHITE BREADS|        1|  WEGMAN WHITE BREAD|        1|          010101| BAKESHOP|       1|WG HOMESTYLE BREAD|      60505|        16.0|            OZ|
|      STRM| VENDOR WHITE BREADS|        2|STROEHMANN WHITE ...|       30|          010230| BAKESHOP|       1|     STRH KING BRD|      58944|        2

# Task 2.7 | 7 items fall into the WHOLE MILK class

In [10]:
itemDF.filter(itemDF['class_name'] == 'WHOLE MILK').count()

7

In [ ]:
customerDF.show(1)

+-----------+--------------------+-----------+----------------+---------------+-----------------+-----------+---------+-------+----------+-------+----------------+---------------+---------------+
|adult_count|    application_date|bad_address|birth_yr_head_hh|birth_yr_oldest|birth_yr_youngest|child_count|hh_income|hh_size|hshld_acct|privacy|wine_email_click|wine_email_open|wine_email_sent|
+-----------+--------------------+-----------+----------------+---------------+-----------------+-----------+---------+-------+----------+-------+----------------+---------------+---------------+
|          2|2012-10-21 00:00:...|          N|            1981|              0|                0|          0|    62500|      2|       559|      N|               0|              0|              0|
+-----------+--------------------+-----------+----------------+---------------+-----------------+-----------+---------+-------+----------+-------+----------------+---------------+---------------+
only showing top 1 r

# Task 2.11 | Average Household Size of Customers for Every Store

In [ ]:
from pyspark.sql import functions as F
postransDF.join(customerDF, 'hshld_acct') \
    .filter(customerDF.hshld_acct>0) \
    .groupBy('store_num') \
    .agg(F.avg('hh_size').alias('average household size')) \
    .join(storeDF, 'store_num') \
    .select('store_name','average household size') \
    .sort('average household size',ascending=False) \
    .show()